In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as mpf


In [96]:
df = pd.read_csv("market-data.csv")
df = df[::-1]  
df 

,timestamp,open,high,low,close
99,2025-03-28 16:00:00,469.020,492.7535,435.5720,466.700
98,2025-03-28 17:00:00,466.685,487.1613,435.5720,467.250
97,2025-03-28 18:00:00,467.250,468.9400,467.0300,467.180
96,2025-03-28 19:00:00,467.155,467.3500,466.7400,466.890
95,2025-03-31 04:00:00,464.660,464.6600,462.8100,462.840
...,...,...,...,...,...
4,2025-04-07 15:00:00,423.150,426.2600,420.1450,423.950
3,2025-04-07 16:00:00,423.960,447.2455,422.0000,425.550
2,2025-04-07 17:00:00,425.550,447.2455,422.2278,426.700
1,2025-04-07 18:00:00,426.700,428.5800,425.9400,428.559


In [97]:
# Step 1: Parse timestamp and keep as column
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

# Step 2: Create a 'Date' column for grouping
df['Date'] = df['timestamp'].dt.date  # keep timestamp, extract date separately

# Step 3: Group to daily OHLC
daily = df.groupby('Date').agg({
    'high': 'max',
    'low': 'min',
    'open': 'first',
    'close': 'last'
})

# Step 4: Shift to get previous day's high/low
daily['Prev_High'] = daily['high'].shift(1)
daily['Prev_Low'] = daily['low'].shift(1)

daily.reset_index(inplace=True)

# Step 5: Merge previous day high/low into main df
df = df.merge(daily[['Date', 'Prev_High', 'Prev_Low']], on='Date', how='left')

# Step 6: Filter for rows with complete previous day data
filtered = df[df['Prev_High'].notnull() & df['Prev_Low'].notnull()]

# ✅ Final Output
print(filtered.head())


            timestamp    open     high     low   close        Date  Prev_High  \
4 2025-03-31 04:00:00  464.66  464.660  462.81  462.84  2025-03-31   492.7535   
5 2025-03-31 05:00:00  462.81  463.590  461.89  462.52  2025-03-31   492.7535   
6 2025-03-31 06:00:00  462.65  463.480  462.31  463.05  2025-03-31   492.7535   
7 2025-03-31 07:00:00  463.13  463.460  460.71  462.05  2025-03-31   492.7535   
8 2025-03-31 08:00:00  463.36  469.291  460.77  463.22  2025-03-31   492.7535   

   Prev_Low  
4   435.572  
5   435.572  
6   435.572  
7   435.572  
8   435.572  


In [98]:
# Step 1: Convert to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Step 2: Make a temporary index for time-based operations
df = df.sort_values('timestamp').reset_index(drop=True)
df['datetime'] = df['timestamp']  # copy for indexing

# Step 3: Set datetime as index temporarily
df = df.set_index('datetime')

# Step 4: Localize to UTC only if tz-naive
if df.index.tz is None:
    df.index = df.index.tz_localize("UTC")

# Step 5: Convert to Eastern (NY time)
df.index = df.index.tz_convert("US/Eastern")

# Step 6: Create date column for grouping
df['Date'] = df.index.date

# Step 7: Filter London session: 3am–7am
london_session = df.between_time("03:00", "07:00")

# Step 8: Aggregate highs/lows per London session
london_range = london_session.groupby(london_session.index.date).agg({
    'high': 'max',
    'low': 'min'
}).rename(columns={'high': 'London_High', 'low': 'London_Low'})

# Step 9: Prepare to merge
london_range.reset_index(inplace=True)
london_range.rename(columns={'index': 'Date'}, inplace=True)

# Step 10: Merge into original df (which still has 'timestamp')
df = df.reset_index()  # bring datetime back to column, keep original timestamp too
df.drop(columns=['London_High', 'London_Low'], errors='ignore', inplace=True)
df = df.merge(london_range, on='Date', how='left')
df.drop(columns='Date', inplace=True)

df

,datetime,timestamp,open,high,low,close,Prev_High,Prev_Low,London_High,London_Low
0,2025-03-28 12:00:00-04:00,2025-03-28 16:00:00,469.020,492.7535,435.5720,466.700,NaN,NaN,NaN,NaN
1,2025-03-28 13:00:00-04:00,2025-03-28 17:00:00,466.685,487.1613,435.5720,467.250,NaN,NaN,NaN,NaN
2,2025-03-28 14:00:00-04:00,2025-03-28 18:00:00,467.250,468.9400,467.0300,467.180,NaN,NaN,NaN,NaN
3,2025-03-28 15:00:00-04:00,2025-03-28 19:00:00,467.155,467.3500,466.7400,466.890,NaN,NaN,NaN,NaN
4,2025-03-31 00:00:00-04:00,2025-03-31 04:00:00,464.660,464.6600,462.8100,462.840,492.7535,435.572,469.291,457.33
...,...,...,...,...,...,...,...,...,...,...
95,2025-04-07 11:00:00-04:00,2025-04-07 15:00:00,423.150,426.2600,420.1450,423.950,450.6600,419.690,443.140,396.00
96,2025-04-07 12:00:00-04:00,2025-04-07 16:00:00,423.960,447.2455,422.0000,425.550,450.6600,419.690,443.140,396.00
97,2025-04-07 13:00:00-04:00,2025-04-07 17:00:00,425.550,447.2455,422.2278,426.700,450.6600,419.690,443.140,396.00
98,2025-04-07 14:00:00-04:00,2025-04-07 18:00:00,426.700,428.5800,425.9400,428.559,450.6600,419.690,443.140,396.00
